In [1]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [2]:
symbtr_cols = ['Sira', 'Kod', 'Nota53', 'NotaAE', 'Koma53', 'KomaAE', 
               'Pay', 'Payda', 'Ms', 'LNS', 'Bas', 'Soz1', 'Offset']

In [3]:
symbTr = 'makam--form--usul--name--composer'

symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

savefile = os.path.join('..', '..', 'symbtr_corrections', 'usul', symbTr + '.txt')

In [4]:
# extract symbtr data
txtdata = extractor.extract(symbtrtxtfile)[0]
mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

# merge
data = extractor.merge(txtdata, mu2header)

    51.0: yâr   and Ay  are in the same measure!
    47.0: sı   and Ö are in the same measure!
    23.0: yâr   and Ay  are in the same measure!
    19.0: sı   and Ö are in the same measure!
buselik--turku--yuruksemai_ii--igdirin_al--igdir, 128, VOCAL_SECTION does not start on a measure: 19.3333333333
buselik--turku--yuruksemai_ii--igdirin_al--igdir, 155, VOCAL_SECTION does not start on a measure: 23.3333333333
buselik--turku--yuruksemai_ii--igdirin_al--igdir, 319, VOCAL_SECTION does not start on a measure: 47.3333333333
buselik--turku--yuruksemai_ii--igdirin_al--igdir, 346, VOCAL_SECTION does not start on a measure: 51.3333333333
   Unknown title target.


In [5]:
variant = {}
for vrt in usuldict[data['usul']['symbtr_slug']]['variants']:
    if vrt['mu2_name'] == data['usul']['mu2_name']:
        variant = vrt
        break
        

In [7]:
df = pd.read_csv(symbtrtxtfile, sep='\t')
if not df.iloc[0]['Kod'] == 51:
    # create the usul row
    # 1	51			0	0	zaman	mertebe	0	usul_symbtr_internal_id	0	usul_mu2_name	0
    # 1	51			0	0	6	4	0	90	0	Yürüksemâî (6/4)	0
    usulrow = pd.DataFrame({'Sira':1, 'Kod':51, 'Nota53':'', 'NotaAE':'', 'Koma53':'', 'KomaAE':'',
                            'Pay':int(variant['num_pulses']),'Payda':int(variant['mertebe']),'Ms':0,'Offset':0,
                            'LNS':variant['symbtr_internal_id'],'Bas':0,'Soz1':variant['mu2_name']}, index=[0])
    df2 = pd.DataFrame(df,index=range(1, df.size+1))
    
    for index, row in df.iterrows():       
        # change null to empty string
        for key, val in row.iteritems():
            if pd.isnull(val):
                row[key] = ''

        # make sure that "Sira" column continues consecutively
        row['Sira'] = row['Sira']+1

        # reassign
        df.iloc[index] = row
        
    df_usul = pd.concat([usulrow, df], ignore_index=True)[symbtr_cols]
    df_usul.to_csv(savefile, sep='\t', index=False, encoding='utf-8')
else:
    print symbTr + " starts with the usul row"